In [2]:
import pyaudio
from deepgram import DeepgramClient, LiveOptions, LiveTranscriptionEvents
import threading
from datetime import datetime
from dotenv import load_dotenv
import json

load_dotenv()

# Audio settings
FORMAT = pyaudio.paInt16
CHANNELS = 1  # Mono
RATE = 44100  # Sample rate
CHUNK = 1024  # Buffer size in frames

def main():
    global transcriptions
    try:
        # Add a list to store all transcriptions
        transcriptions = []

        # Initialize PyAudio and Deepgram
        audio = pyaudio.PyAudio()
        deepgram = DeepgramClient()
        dg_connection = deepgram.listen.websocket.v("1")

        # Add connection status handlers with correct signatures
        def on_open(self, connection):
            print("Connected to Deepgram!")

        def on_close(self, connection):
            print("Disconnected from Deepgram!")

        def on_error(self, error):
            print(f"Error from Deepgram: {error}")

        def on_message(self, result):
            try:
                # Convert result to dictionary if it isn't already
                if isinstance(result, str):
                    result = json.loads(result)
                
                if result.get('is_final'):
                    transcript_data = {
                        'timestamp': datetime.now().isoformat(),
                        'transcript': result['channel']['alternatives'][0]['transcript'],
                        'confidence': result['channel']['alternatives'][0]['confidence'],
                        'words': result['channel']['alternatives'][0]['words'],
                        'start_time': result['start'],
                        'duration': result['duration'],
                        'request_id': result['metadata']['request_id']
                    }
                    
                    # Store the transcription
                    transcriptions.append(transcript_data)
                    
                    # Print current transcription
                    print(f"[{transcript_data['timestamp']}] Transcription: {transcript_data['transcript']}")
                    
            except Exception as e:
                print(f"Error in transcription callback: {e}")
                print(f"Result type: {type(result)}")
                print(f"Result content: {result}")

        # Register all event handlers
        dg_connection.on(LiveTranscriptionEvents.Open, on_open)
        dg_connection.on(LiveTranscriptionEvents.Close, on_close)
        dg_connection.on(LiveTranscriptionEvents.Error, on_error)
        dg_connection.on(LiveTranscriptionEvents.Transcript, on_message)

        # Start Deepgram connection with more detailed error handling
        options = LiveOptions(
            model="nova-2",
            encoding="linear16",
            channels=CHANNELS,
            sample_rate=RATE
        )
        
        if dg_connection.start(options) is False:
            print("Failed to start Deepgram connection")
            return

        # Start audio stream
        stream = audio.open(format=FORMAT,
                          channels=CHANNELS,
                          rate=RATE,
                          input=True,
                          frames_per_buffer=CHUNK)

        print("\nRecording... Press Enter to stop.\n")

        # Create a flag for stopping the recording
        should_stop = threading.Event()

        # Define audio capture thread
        def capture_audio():
            while not should_stop.is_set():
                data = stream.read(CHUNK)
                dg_connection.send(data)

        # Start the capture thread
        audio_thread = threading.Thread(target=capture_audio)
        audio_thread.start()

        # Modify the cleanup section to save transcriptions
        def save_transcriptions():
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"transcription_{timestamp}.json"
            with open(filename, 'w') as f:
                json.dump(transcriptions, f, indent=2)
            print(f"Transcriptions saved to {filename}")

        # Wait for Enter key
        input("")
        should_stop.set()

        # Clean up
        audio_thread.join()
        stream.stop_stream()
        stream.close()
        audio.terminate()
        dg_connection.finish()
        
        # Save transcriptions before exiting
        save_transcriptions()
        print("Finished recording")

    except Exception as e:
        print(f"Error: {e}")
        return

if __name__ == "__main__":
    main()

Connected to Deepgram!

Recording... Press Enter to stop.

Error in transcription callback: 'LiveResultResponse' object has no attribute 'get'
Result type: <class 'deepgram.clients.listen.v1.websocket.response.LiveResultResponse'>
Result content: {
    "channel": {
        "alternatives": [
            {
                "transcript": "right so we're given a test to",
                "confidence": 0.99609375,
                "words": [
                    {
                        "word": "right",
                        "start": 1.36,
                        "end": 1.76,
                        "confidence": 0.7260742
                    },
                    {
                        "word": "so",
                        "start": 1.76,
                        "end": 2.08,
                        "confidence": 0.99902344
                    },
                    {
                        "word": "we're",
                        "start": 2.08,
                        "end": 2.24,
    

In [3]:
transcriptions

[]